In [1]:
import sys
sys.path.append("./")
sys.path.append("./hw2/stud/")
from torch.utils.data import DataLoader
import config
# from tqdm.auto import tqdm
import numpy as np
from utils import *
from load import *
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning import Trainer
from glossBERT import GlossBERT


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# FINE_TRAIN_DATA
# FINE_VAL_DATA
# FINE_TEST_DATA


In [3]:
print(config.DEVICE)

cuda


In [4]:
seed_everything(config.SEED)

In [5]:

# train_dataset = CoarseGrainedDataset(config.COARSE_TRAIN_DATA)
# val_dataset = CoarseGrainedDataset(config.COARSE_VAL_DATA)
# test_dataset = CoarseGrainedDataset(config.COARSE_TEST_DATA)

# train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# mapping = load_map(config.MAP_PATH)

In [6]:

train_dataset = FineGrainedDataset(config.FINE_TRAIN_DATA)
val_dataset = FineGrainedDataset(config.FINE_VAL_DATA)
test_dataset = FineGrainedDataset(config.FINE_TEST_DATA)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, collate_fn=glossBERT_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.BATCH_SIZE, shuffle=False, collate_fn=glossBERT_collate_fn)

mapping = load_map(config.MAP_PATH)


In [7]:
# take a look at the data 
#load one data fr om the train_dataset
print(train_dataset[0])

(['Bailly', ',', 'after', 'leaving', 'Fort Snelling', 'in', 'August', '1821', ',', 'was', 'forced', 'to', 'leave', 'some', 'of', 'the', 'cattle', 'at', 'the', "Hudson's Bay Company's", 'post', 'on', 'Lake Traverse', '``', 'in', 'the', 'Sieux Country', "''", 'and', 'reached', 'Fort Garry', ',', 'as', 'the', 'Selkirk', "Hudson's Bay Company", 'center', 'was', 'now', 'called', ',', 'late', 'in', 'the', 'fall', '.', 'He', 'set out', 'on', 'his', '700', '-', '"mile"', 'return', 'journey', 'with', 'five', 'families', 'of', 'discontented', 'and', 'disappointed', 'Swiss', 'who', 'turned', 'their', 'eyes', 'toward', 'the', 'United States', '.', '[SEP]', 'mile:', 'a', 'unit', 'of', 'length', 'used', 'in', 'navigation;', 'exactly', '1,852', 'meters;', 'historically', 'based', 'on', 'the', 'distance', 'spanned', 'by', 'one', 'minute', 'of', 'arc', 'in', 'latitude', '[SEP]'], ['person', ',', 'after', 'leave', 'location', 'in', 'august', '1821', ',', 'be', 'force', 'to', 'leave', 'some', 'of', 'the'

In [8]:
# # # display one item
# for batch in train_loader:
#     input_ids, senses_ids, labels, attention_mask = batch
#     # pritn one item
#     print(input_ids)
#     print(senses_ids)
#     print(labels)
#     print(attention_mask)

#     break



In [ ]:
model = GlossBERT()
# model.to(config.DEVICE)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# TRAINER
# wandb_logger = WandbLogger(project='nlp_hw2', log_model=True)
trainer = Trainer(gpus=0, max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])#, logger=wandb_logger)
trainer.fit(model, train_loader, val_loader)
# trainer = Trainer(max_epochs=10, callbacks=[ModelCheckpoint(monitor='val_loss')])
# trainer.fit(model, train_loader, val_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1566: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name       | Type             | Params
------------------------------------------------
0 | bert       | RobertaModel     | 124 M 
1 | relu       | ReLU             | 0     
2 | classifier | Linear           | 1.5 K 
3 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
498.589   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/5909 [00:00<?, ?it/s] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
Epoch 0:   0%|          | 1/5909 [03:37<357:19:07, 217.73s/it, loss=0.638, v_num=1, train_loss=0.638, train_acc=0.750]tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Epoch 0:   0%|          | 2/5909 [03:52<190:53:56, 116.34s/it, loss=0.589, v_num=1, train_loss=0.539, train_acc=0.938]tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1])
Epoch 0:   0%|          | 3/5909 [04:01<132:07:30, 80.54s/it, loss=0.604, v_num=1, train_loss=0.634, train_acc=0.688] tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 

/home/francesco/miniconda3/envs/nlp2023-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:682: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  return trainer_fn(*args, **kwargs)
